In [77]:
import ddddocr
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pickle
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
import io
from PIL import Image
import time
import argparse

In [78]:
url='https://www.famiticket.com.tw/Home/User/SignIn'
event = 'T1 臺北戰神2023-24主場例行賽'
options = Options()

options.add_argument("--incognito")
driver = uc.Chrome(options=options)
driver.get(url)


In [79]:
with open('./famiticket_credentials') as f:
    email, password = [cred.strip().split('=')[-1] for cred in f.readlines()]

In [80]:
email_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='usr_act']")))
email_input.clear()
email_input.send_keys(email)
password_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[id='usr_pwd']")))
password_input.clear()
password_input.send_keys(password)
login_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[id='btnLogin']")))
while(True):
    try:
        login_button.click()
        break
    except:
        pass

In [81]:
event_div_lst= WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='card__overlay-text']")))
for div in event_div_lst:
    if(div.get_attribute('innerHTML').strip() == event):
        matched_event_div = div
while(True):
    matched_event_a = WebDriverWait(matched_event_div, 10).until(EC.presence_of_element_located((By.XPATH, "../../../..")))
    try:
        matched_event_a.click()
        break
    except Exception as e:
        pass


In [82]:
buy_now_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[id='buyWaiting']")))
while(True):
    try:
        buy_now_button.click()
        break
    except:
        pass

In [83]:
event_priority = [
    ['2023.12.30 (六) 14:00', None, None, None]
    ]
wanted_section_keywords_priority = [['']]

In [84]:
event_tr_lst = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "tbody > tr")))

finished_choosing_events = False
while(not finished_choosing_events):
    for event_datetime, event_name, event_combination, event_venue in event_priority:
        print('Now trying', event_datetime, event_name, event_combination, event_venue)
        matched_event_idx = None
        for i, event_tr in enumerate(event_tr_lst):
            event_td_lst = WebDriverWait(event_tr, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "td")))
            datetime_match, name_match, combination_match, venue_match = True, True, True, True
            if(event_datetime is not None):
                datetime_match = (event_td_lst[0].text == event_datetime)
            if(event_name is not None):
                name_match = (event_td_lst[1].text == event_name)
            if(event_combination is not None):
                combination_match = (event_td_lst[2].text == event_venue)
            if(event_venue is not None):
                venue_match = (event_td_lst[3].text == event_venue)
            if(datetime_match and name_match and venue_match):
                matched_event_idx = i
                matched_event_status_td = event_td_lst[-1]
                break
        if(matched_event_idx is not None):
            current_status = matched_event_status_td.text
            if(current_status.find('立即購買') != -1):
                print('Found matched event', event_datetime, event_name, event_combination, event_venue)
                while(True):
                    try:
                        matched_event_status_td.click()
                        finished_choosing_events = True
                        break
                    except Exception as e:
                        continue
            elif(current_status.find('Sale starts at') != -1):
                print(current_status, ', refreshing...', sep='')
                time.sleep(.5)
                driver.refresh()
                break
            elif(current_status.find('Sale ended on') != -1):
                continue
        else:
            print('No matched events, should abort')
            driver.refresh()
            continue
        if(finished_choosing_events):
            break


TimeoutException: Message: 
